In [ ]:
# Import basic modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
plt.style.use('ggplot')

##### RF-based models
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from joblib import dump, load
from tqdm import tqdm
from sklearn.model_selection import RandomizedSearchCV,StratifiedKFold,GridSearchCV,cross_val_score,KFold
import xgboost as xgb
import joblib

########## Quantile Regression Models
from quantile_forest import RandomForestQuantileRegressor

##### For SARIMAX and TBATS models
from pmdarima import auto_arima
## For outliers detection
from sklearn import preprocessing, svm
## For stationarity test and decomposition
import statsmodels.tsa.api as smt
import statsmodels.api as sm
import holidays
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

### XGBoost
import xgboost as xgb

# parameter tuning
from sklearn.model_selection import TimeSeriesSplit

import time
import joblib
import pickle
# start_time = time.time()
# main()
# print("--- %s seconds ---" % (time.time() - start_time))

##### Import Dataset
df_train = pd.read_csv('Data/TW_use_case/data_train_15min.csv')
df_test = pd.read_csv('Data/TW_use_case/data_test_15min.csv')


In [2]:
train_days = df_train.day.unique()
test_days = df_test.day.unique()

In [3]:
'combine df_train and df_test'
df = pd.concat([df_train, df_test], axis=0)
df = df.reset_index(drop=True)
df.columns = ['vendor_geohash', 'day', 'hour', 'per_15min', 'counts', 'day_of_week']

In [4]:
######### Util ##########
def dataset_extraction(df):
    'extracting dataset for each zone'
    appended_data = []
    df = df.sort_values(by = ['vendor_geohash','day','hour','per_15min'])
    original_index = df.index
    grid_list = df.vendor_geohash.unique()
    for grid in grid_list:
        df_zone = df[df['vendor_geohash'] == grid]
        df_zone['AR1'] = df_zone['counts'].shift(1)
        df_zone['AR2'] = df_zone['counts'].shift(2)
        df_zone['AR3'] = df_zone['counts'].shift(3)
        df_zone['AR4'] = df_zone['counts'].shift(4)
        appended_data.append(df_zone)
    appended_data = pd.concat(appended_data)
    appended_data = appended_data.loc[original_index]
    return appended_data

def save_model(model,model_name,zone_name,model_type):
    'model saving'
    if model_type == 'RF':
        # file_name = f'new_models/{model_name}_{zone_name}'
        file_name = f'Models/{model_name}_{zone_name}'
        dump(model, f'{file_name}.joblib')
    
    elif model_type == 'XGB':
        file_name = f'Models/{model_name}_{zone_name}'
        dump(model, f'{file_name}.joblib')
        
    elif model_type == 'QRF':
        file_name = f'Models/{model_name}_{zone_name}'
        dump(model, f'{file_name}.joblib')

    elif model_type == 'SARIMA':
        file_name = f'Models/{model_name}_{zone_name}'
        joblib.dump(model, f'{file_name}.pkl')
    
    elif model_type == 'SARIMAX':
        file_name = f'Models/{model_name}_{zone_name}'
        joblib.dump(model, f'{file_name}.pkl')

    elif model_type == 'TBATS':
        file_name = f'Models/{model_name}_{zone_name}.sav'
        pickle.dump(model, open(file_name, 'wb'))
    else:
        pass


def generate_predict(model, test_X, model_type = 'RF'):
    'generate one-step ahead prediction'
    if model_type == 'RF':
        yhat = model.predict(test_X)
    else:
        pass

def mean_absolute_scaled_error(y_true, y_pred, y_naive):
    """
    Calculate the Mean Absolute Scaled Error (MASE) for a forecasting model.

    Parameters:
    - y_true: Array of actual observed values.
    - y_pred: Array of forecasted values from the model being evaluated.
    - y_naive: Array of forecasted values from a naive (benchmark) model.

    Returns:
    - MASE: Mean Absolute Scaled Error value.
    """
    # Calculate Mean Absolute Error (MAE) for the model
    mae_model = np.mean(np.abs(y_true - y_pred))
    # Calculate Mean Absolute Error (MAE) for the naive (benchmark) model
    mae_naive = np.mean(np.abs(y_true - y_naive))
    # Calculate MASE
    mase = mae_model / mae_naive 
    return mase


def evaluation(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred).round(3)
    rmse = mean_squared_error(y_true, y_pred, squared=False).round(3)
    r2 = r2_score(y_true, y_pred).round(3)
    return mae, rmse, r2


In [5]:
'create complete dataset'
df = dataset_extraction(df)
print(len(df))
print(sum(df.counts))
# df.head()

216000
1996283.0


In [6]:
'Splitting dataset into train and test'
# drop nan values
df = df.dropna()
df_train = df[df.day.isin(train_days)]
df_test = df[df.day.isin(test_days)]

In [8]:
# 'check NAN values'
# print(df_train.isnull().sum())
# print(df_test.isnull().sum())

## Models Training

In [ ]:
'Model modules'
def RandomForest_predictor_with_hyperparam_tuning(df_train, df_test, model_name='LDRF'):
    # define hyperparameters
    param_grid_ = {
    'n_estimators': [int(x) for x in np.linspace(start = 50, stop = 200, num = 6)],
    'max_depth': [None, 3,4,5,6],
    'min_samples_split': [4, 6, 8, 10],
    'min_samples_leaf': [2, 3,5,7,10],
    'max_features' : ['auto', 'sqrt'],
    'bootstrap' : [True, False]}

    # fit a model per grid
    for grid in tqdm(df_train.vendor_geohash.unique()):
        print('Training model for', grid)
        df_train_zone = df_train[df_train.vendor_geohash == grid]
        df_test_zone = df_test[df_test.vendor_geohash == grid]
        if model_name == 'LDRF':
            X_train, y_train = df_train_zone[['day_of_week','hour','AR1','AR2','AR3','AR4']], df_train_zone['counts']
            X_test, y_test = df_test_zone[['day_of_week','hour','AR1','AR2','AR3','AR4']], df_test_zone['counts']
        else:
            X_train, y_train = df_train_zone[['day_of_week','hour']], df_train_zone['counts']
            X_test, y_test = df_test_zone[['day_of_week','hour']], df_test_zone['counts']
        
        # define random forest regressor
        rf = RandomForestRegressor(random_state=42)
        
        # Create time series cross-validation
        tscv = TimeSeriesSplit(n_splits=10)
        # create RandomizedSearchCV
        gscv = RandomizedSearchCV(estimator=rf, 
                                    param_distributions=param_grid_, 
                                    cv=tscv, n_jobs=6, verbose=0,
                                    n_iter=200, random_state=39,
                                    scoring='neg_root_mean_squared_error')

        start_time = time.time()
        gscv.fit(X_train, y_train)
        train_time_ = time.time() - start_time
        print('Total parameter tuning time:', train_time_)
        # Get the best hyperparameters
        best_params = gscv.best_params_
        print(f"{grid} -- Best Hyperparameters:", best_params)
        start_train_time = time.time()
        best_model = RandomForestRegressor(n_estimators= best_params['n_estimators'], 
                                            min_samples_split= best_params['min_samples_split'], 
                                            min_samples_leaf = best_params['min_samples_leaf'], 
                                            max_features= best_params['max_features'], 
                                            max_depth= best_params['max_depth'], 
                                            bootstrap= best_params['bootstrap'], random_state=42)
        best_model.fit(X_train, y_train)
        end_train_time = time.time() - start_train_time
        print(f"Best Model Training Time : {end_train_time}")
        
        print(f'\n ** Training Set Results ** \n')
        y_pred = best_model.predict(X_train)
        mae, rmse, r2 = evaluation(y_train, y_pred)
        print('MAE:', mae, 'RMSE:', rmse, 'R2:', r2)

        print(f'\n ** Testing Set Results ** \n')
        y_pred = best_model.predict(X_test)
        mae, rmse, r2 = evaluation(y_test, y_pred)
        print('MAE:', mae, 'RMSE:', rmse, 'R2:', r2)

        print('Model saved for', grid)
        save_model(best_model, model_name, grid, model_type='RF')

In [ ]:
def XGBoost_predictor_with_hyperparam_tuning(df_train, df_test, model_name='LDXGB'):
    'an xgboost model with hyperparameter tuning'
    # define hyperparameters
    param_grid_ = {'objective' : ["reg:squarederror"], 
                'n_estimators' : [int(x) for x in np.linspace(start = 50, stop = 200, num = 10)],
                'learning_rate': [0.01, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3],
                'max_depth': [int(x) for x in np.linspace(3, 6, num = 3)],
                'subsample':[0.5,0.75,1]
                }
    # fit a model per grid
    for grid in tqdm(df_train.vendor_geohash.unique()):
        print('Training model for', grid)
        xgb_regressor = xgb.XGBRegressor()
        df_train_zone = df_train[df_train.vendor_geohash == grid]
        df_test_zone = df_test[df_test.vendor_geohash == grid]
        if model_name == 'LDXGB':
            X_train, y_train = df_train_zone[['day_of_week','hour','AR1','AR2','AR3','AR4']], df_train_zone['counts']
            X_test, y_test = df_test_zone[['day_of_week','hour','AR1','AR2','AR3','AR4']], df_test_zone['counts']
        else:
            X_train, y_train = df_train_zone[['day_of_week','hour']], df_train_zone['counts']
            X_test, y_test = df_test_zone[['day_of_week','hour']], df_test_zone['counts']
        
        # Create time series cross-validation
        tscv = TimeSeriesSplit(n_splits=10)
        # kf = KFold(n_splits=10, shuffle=False)
            # Create GridSearchCV with the Random Forest Regressor and hyperparameter grid
        gscv = RandomizedSearchCV(estimator=xgb_regressor, param_distributions=param_grid_, 
                                scoring='neg_root_mean_squared_error', cv=tscv, 
                                n_iter = 200, verbose=2, n_jobs = 6, random_state=42)
        start_time = time.time()
        gscv.fit(X_train, y_train)
        train_time_ = time.time() - start_time
        print('Total parameter tuning time:', train_time_)
        # Get the best hyperparameters
        best_params = gscv.best_params_
        print(f"{grid} -- Best Hyperparameters:", best_params)

        start_train_time = time.time()
        best_model = xgb.XGBRegressor(objective = "reg:squarederror", 
                                    n_estimators = best_params['n_estimators'], 
                                    learning_rate = best_params['learning_rate'], 
                                    max_depth = best_params['max_depth'], 
                                    subsample = best_params['subsample'],
                                    nthread = 4, random_state=42)
        best_model.fit(X_train, y_train)
        end_train_time = time.time() - start_train_time
        print(f"Best Model Training Time : {end_train_time}")

        print(f'\n ** Training Set Results ** \n')
        y_pred = best_model.predict(X_train)
        mae, rmse, r2 = evaluation(y_train, y_pred)
        print('MAE:', mae, 'RMSE:', rmse, 'R2:', r2)

        print(f'\n ** Testing Set Results ** \n')
        y_pred = best_model.predict(X_test)
        mae, rmse, r2 = evaluation(y_test, y_pred)
        print('MAE:', mae, 'RMSE:', rmse, 'R2:', r2)

        print('Model saved for', grid)
        save_model(best_model, model_name, grid, model_type='XGB')

In [ ]:
def ARQRF_predictor_with_hyperparam_tuning(df_train, df_test, model_name='ARQRF'):
    # define hyperparameters
    param_grid_ = {
    'n_estimators': [int(x) for x in np.linspace(start = 50, stop = 200, num = 6)],
    'max_depth': [None, 3,4,5,6],
    'min_samples_split': [4, 6, 8, 10],
    'min_samples_leaf': [2,3,5,7,10],
    'max_features' : ['auto', 'sqrt'],
    'bootstrap' : [True, False]}
    for grid in tqdm(df_train.vendor_geohash.unique()):
        print('Training model for', grid)
        arqrf = RandomForestQuantileRegressor()
        df_train_zone = df_train[df_train.vendor_geohash == grid]
        df_test_zone = df_test[df_test.vendor_geohash == grid]
        if model_name == 'ARQRF':
            X_train, y_train = df_train_zone[['day_of_week','hour','AR1','AR2','AR3','AR4']], df_train_zone['counts']
            X_test, y_test = df_test_zone[['day_of_week','hour','AR1','AR2','AR3','AR4']], df_test_zone['counts']
        else:
            X_train, y_train = df_train_zone[['day_of_week','hour']], df_train_zone['counts']
            X_test, y_test = df_test_zone[['day_of_week','hour']], df_test_zone['counts']
    
        # Create time series cross-validation
        tscv = TimeSeriesSplit(n_splits=10)
        # Random search of parameters, using 10 fold cross validation,
        gscv = RandomizedSearchCV(estimator = arqrf,
                                            param_distributions = param_grid_,
                                            n_iter = 200, cv = tscv,
                                            verbose=2, n_jobs = 6)
        start_time = time.time()
        gscv.fit(X_train, y_train)
        train_time_ = time.time() - start_time
        print('Total parameter tuning time:', train_time_)
        # Get the best hyperparameters
        best_params = gscv.best_params_
        print(f"{grid} -- Best Hyperparameters:", best_params)

        start_train_time = time.time()
        best_model = RandomForestQuantileRegressor(n_estimators= best_params['n_estimators'], 
                                            min_samples_split= best_params['min_samples_split'], 
                                            min_samples_leaf = best_params['min_samples_leaf'], 
                                            max_features= best_params['max_features'], 
                                            max_depth= best_params['max_depth'], 
                                            bootstrap= best_params['bootstrap'])
        best_model.fit(X_train, y_train)
        end_train_time = time.time() - start_train_time
        print(f"Best Model Training Time : {end_train_time}")

        print(f'\n ** Training Set Results ** \n')
        y_pred = best_model.predict(X_train)
        mae, rmse, r2 = evaluation(y_train, y_pred)
        print('MAE:', mae, 'RMSE:', rmse, 'R2:', r2)

        print(f'\n ** Testing Set Results ** \n')
        y_pred = best_model.predict(X_test)
        mae, rmse, r2 = evaluation(y_test, y_pred)
        print('MAE:', mae, 'RMSE:', rmse, 'R2:', r2)

        print('Model saved for', grid)
        save_model(best_model, model_name, grid, model_type='QRF')


In [10]:
from tbats import TBATS, BATS
def TBATS_training(df_train, df_test, model_name='TBATS'):
    'takes a long time to train'
    for grid in tqdm(df_train.vendor_geohash.unique()):
        print('Training model for', grid)
        df_train_zone = df_train[df_train.vendor_geohash == grid]
        df_test_zone = df_test[df_test.vendor_geohash == grid]
        start_train_time = time.time()
        estimator = TBATS(seasonal_periods=(96, 672)) # 96 for daily seasonality and 672 for weekly seasonality
        model = estimator.fit(df_train_zone['counts'][-2688:])
        end_train_time = time.time() - start_train_time
        print(f"Best Model Training Time : {end_train_time}")

        print(f'\n ** Testing Set Results ** \n')
        y_pred = model.forecast(steps=len(df_test_zone))
        mae, rmse, r2 = evaluation(df_test_zone['counts'], y_pred)
        print('MAE:', mae, 'RMSE:', rmse, 'R2:', r2)

        print('Model saved for', grid)
        save_model(model, model_name, grid, model_type='TBATS')


In [11]:
def SARIMA_training(df_train, df_test, model_name='SARIMA'):
    'takes a long time to train'
    for grid in tqdm(df_train.vendor_geohash.unique()):
        print('Training model for', grid)
        df_train_zone = df_train[df_train.vendor_geohash == grid][-2688:]
        df_test_zone = df_test[df_test.vendor_geohash == grid]
        start_train_time = time.time()
        model = auto_arima(df_train_zone['counts'], seasonal=True, 
                           m=96, stepwise=False, 
                           suppress_warnings=True, n_jobs=4)
        end_train_time = time.time() - start_train_time
        print(f"Best Model Training Time : {end_train_time}")

        print(f'\n ** Testing Set Results ** \n')
        y_pred = model.predict(n_periods=len(df_test_zone))
        mae, rmse, r2 = evaluation(df_test_zone['counts'], y_pred)
        print('MAE:', mae, 'RMSE:', rmse, 'R2:', r2)

        print('Model saved for', grid)
        save_model(model, model_name, grid, model_type='SARIMA')


In [12]:
def SARIMAX_training(df_train, df_test, model_name='SARIMA'):
    for grid in tqdm(df_train.vendor_geohash.unique()):
        print('Training model for', grid)
        df_train_zone = df_train[df_train.vendor_geohash == grid][-2688:]
        df_test_zone = df_test[df_test.vendor_geohash == grid]
        start_train_time = time.time()
        model = auto_arima(df_train_zone['counts'], exogenous=df_train_zone[['day_of_week','hour']], seasonal=True, m=96, stepwise=True, trace=True)
        end_train_time = time.time() - start_train_time
        print(f"Best Model Training Time : {end_train_time}")

        print(f'\n ** Testing Set Results ** \n')
        y_pred = model.predict(n_periods=len(df_test_zone), exogenous=df_test_zone[['day_of_week','hour']])
        mae, rmse, r2 = evaluation(df_test_zone['counts'], y_pred)
        print('MAE:', mae, 'RMSE:', rmse, 'R2:', r2)

        print('Model saved for', grid)
        save_model(model, model_name, grid, model_type='SARIMAX')

### Training

In [ ]:
TBATS_training(df_train, df_test, model_name='TBATS')

In [ ]:
SARIMA_training(df_train, df_test, model_name='SARIMA')

In [ ]:
SARIMAX_training(df_train, df_test, model_name='SARIMAX')

In [ ]:
ARQRF_predictor_with_hyperparam_tuning(df_train, df_test, model_name='ARQRF')

In [ ]:
XGBoost_predictor_with_hyperparam_tuning(df_train, df_test, model_name='LDXGB')

In [ ]:
RandomForest_predictor_with_hyperparam_tuning(df_train, df_test, model_name='LDRF')